In [1]:
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import tensorflow as tf
from pandas import read_csv
from numpy import array
from numpy import hstack
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils

from sklearn.metrics import mean_squared_error, r2_score

from pandas import read_csv
from datetime import datetime
from math import sqrt
from matplotlib import pyplot
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, Bidirectional
from keras import optimizers

import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import chart_studio.plotly as py
import cufflinks as cf 
cf.go_offline(connected=True)

import seaborn as sns    
import matplotlib as mpl
import matplotlib.font_manager as fm

# 데이터 임포트

In [2]:
data = pd.read_csv('올림픽.csv',
                        skiprows=1, 
                        names=['INDATES','INYEAR', 'INMON', 'INDAY', 'INHOUR', 'OUTDATES', 'OUTYEAR', 'OUTMON', 'OUTDAY', 'OUTHOUR', 'INDATE', 'OUTDATE', 'cnt'],
                        engine='python',
                        encoding ='utf-8-sig')

rawdata = data.copy()

#INDATE, OUTDATE datetime 형식으로 바꾸기
rawdata['INDATE'] = pd.to_datetime(rawdata['INDATE'])
rawdata['OUTDATE'] = pd.to_datetime(rawdata['OUTDATE'])

# 이용시간 구하기
using_time = []
for i in range(len(rawdata.index)):
    using_time.append(rawdata['OUTDATE'][i] - rawdata['INDATE'][i])
    
rawdata['USING'] = list(using_time)

In [3]:
rawdata

,INDATES,INYEAR,INMON,INDAY,INHOUR,OUTDATES,OUTYEAR,OUTMON,OUTDAY,OUTHOUR,INDATE,OUTDATE,cnt,USING
0,2019-11-20,2019,11,20,21,2019-11-20,2019,11,20,21,2019-11-20 21:38:00,2019-11-20 21:40:00,1,0 days 00:02:00
1,2019-11-23,2019,11,23,17,2019-11-23,2019,11,23,17,2019-11-23 17:46:00,2019-11-23 22:29:00,1,0 days 04:43:00
2,2019-11-28,2019,11,28,8,2019-11-28,2019,11,28,8,2019-11-28 08:58:00,2019-11-28 17:08:00,1,0 days 08:10:00
3,2019-11-06,2019,11,6,6,2019-11-06,2019,11,6,6,2019-11-06 06:16:00,2019-11-06 07:07:00,1,0 days 00:51:00
4,2019-11-03,2019,11,3,15,2019-11-03,2019,11,3,15,2019-11-03 15:48:00,2019-11-03 16:23:00,1,0 days 00:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973670,2020-10-29,2020,10,29,21,2020-10-29,2020,10,29,21,2020-10-29 21:00:00,2020-10-29 22:50:00,1,0 days 01:50:00
1973671,2020-10-29,2020,10,29,16,2020-10-29,2020,10,29,16,2020-10-29 16:18:00,2020-10-29 16:23:00,1,0 days 00:05:00
1973672,2020-10-29,2020,10,29,9,2020-10-29,2020,10,29,9,2020-10-29 09:57:00,2020-10-29 13:01:00,1,0 days 03:04:00
1973673,2020-10-29,2020,10,29,12,2020-10-29,2020,10,29,12,2020-10-29 12:10:00,2020-10-29 15:09:00,1,0 days 02:59:00


# 시간별

In [4]:
hour = pd.DataFrame(rawdata.groupby('INHOUR')['USING'].mean(numeric_only=False))
hour

,USING
INHOUR,
0,0 days 06:06:04
1,0 days 03:28:00.034843205
2,0 days 04:21:19.968379446
3,0 days 01:58:14.716981132
4,0 days 01:43:29.115398616
5,0 days 02:42:06.812488315
6,0 days 04:02:11.677718832
7,0 days 05:47:46.161627943
8,0 days 06:03:34.112587173


In [5]:
hour.iplot(kind='bar', title = "올림픽공원 입차 시간별 평균 이용시간", xTitle='Hour', yTitle = '시간', color='skyblue') 

# 일별

In [6]:
day = pd.DataFrame(rawdata.groupby('INDAY')['USING'].mean(numeric_only=False))
day

,USING
INDAY,
1,0 days 02:32:33.279245812
2,0 days 02:47:14.046792603
3,0 days 02:42:46.103080655
4,0 days 02:40:54.898225117
5,0 days 02:36:52.934255529
6,0 days 03:01:50.678698252
7,0 days 02:59:34.776943562
8,0 days 02:52:56.545708304
9,0 days 02:49:10.274647543


In [7]:
day.iplot(kind='bar', title = "올림픽공원 입차 일별 평균 이용시간", xTitle='Day', yTitle = '시간', color='darkslateblue') 

# 월별

In [8]:
mon = pd.DataFrame(rawdata.groupby('INMON')['USING'].mean(numeric_only=False))
mon

,USING
INMON,
1,0 days 02:43:43.524452033
2,0 days 02:53:37.883758566
3,0 days 02:44:56.458121150
4,0 days 02:39:08.264064201
5,0 days 02:40:44.495478980
6,0 days 02:53:20.702783300
7,0 days 02:56:54.956355086
8,0 days 03:00:48.555609487
9,0 days 02:38:49.986579445


In [9]:
mon.iplot(kind='bar', title = "올림픽공원 입차 월별 평균 이용시간", xTitle='MONTH', yTitle = '시간', color='blue') 

# 계절별

In [10]:
sea = pd.DataFrame(rawdata.groupby('INMON')['USING'].mean(numeric_only=False))

sea.reset_index(inplace=True)

se = ['겨울','봄','여름','가을']

#계절로 나누기
win = (sea['USING'][0] + sea['USING'][1] + sea['USING'][11]) / 3
spr = (sea['USING'][2] + sea['USING'][3] + sea['USING'][4]) / 3
summ = (sea['USING'][5] + sea['USING'][6] + sea['USING'][7]) / 3 
aut = (sea['USING'][8] + sea['USING'][9] + sea['USING'][10]) / 3 

season_list = []
season_list.append(win)
season_list.append(spr)
season_list.append(summ)
season_list.append(aut)

table = pd.DataFrame(season_list)
table.index = se

In [11]:
table.iplot(kind='bar', title = "올림픽공원 입차 계절별 평균 이용시간", xTitle='Season', yTitle = '시간', color='powderblue') 